In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [10]:
page_no = 1
job_list = []

while True:
    print(f"Currently scraping: {page_no}")
    url = 'https://everjobs.ug/page/'+str(page_no)+'/'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    page_no += 1

    if page_no >3:
        break
        
    job_table = soup.select('table', id="wpjb-job-list")
    for job in job_table:
        job_info = job.find_all('tr')
        for job in job_info:
            items = {}
            items['Job Title'] = job.find('a').text
            items['Company'] = job.find('small').text.strip()
            items['Location'] = job.find('td',class_="wpjb-column-location").text.strip().split(',')[0]
            items['Job type'] = job.find('td',class_="wpjb-column-location").text.strip().split('\n\n')[-1]
            items['Date'] = job.find('td', class_="wpjb-column-date wpjb-last").text.strip().split('\n')[0]
            items['Recency'] = job.find('td', class_="wpjb-column-date wpjb-last").text.strip().split('\n')[-1]
            items['Link'] = job.find('a').get('href')
            time.sleep(1)
            job_list.append(items)

# Print the number of jobs scraped
print(f"Total jobs scraped: {len(job_list)}")


Currently scraping: 1
Currently scraping: 2
Currently scraping: 3
Total jobs scraped: 40


In [6]:
df = pd.DataFrame(job_list)
df.drop_duplicates(subset = ['Job Title', 'Company', 'Location', 'Date'], inplace = True)
df['Location'].value_counts()

Location
Kampala               365
Mukono                 29
Bugiri                 14
Mityana                12
Soroti                 11
Kakira and Kayunga      7
Luuka                   5
Mbarara                 5
Jinja                   5
Yumbe                   3
Fort Portal             2
Lamwo                   2
Nwoya and Amuru         2
Nwoya                   2
Kyaka                   2
Gulu                    1
Masindi                 1
Bukoto                  1
Bwebajja                1
Adjumani                1
Terego                  1
Kabarole                1
Masaka                  1
Moroto                  1
Kiboga                  1
Kyegegwa                1
Nakivale                1
Tororo                  1
Name: count, dtype: int64

In [8]:
print(f"From: ", df.iloc[-1:, 4])
print(f"To: ", df.iloc[:1, 4])

From:  479    Jan, 13
Name: Date, dtype: object
To:  0    Jan, 20
Name: Date, dtype: object


In [10]:
df.head()

,Job Title,Company,Location,Job type,Date,Recency,Link
0,Chief Finance Officer,NFT Consult LTD,Kampala,Full-time,"Jan, 20",New,https://everjobs.ug/job/chief-finance-officer-88/
1,Quality Control Analyst - Chemistry,Abacus Pharma(A) Limited,Kampala,Full-time,"Jan, 20",New,https://everjobs.ug/job/quality-control-analys...
2,Health Information Assistant,International Rescue Committee (IRC),Yumbe,Contract,"Jan, 20",New,https://everjobs.ug/job/health-information-ass...
3,Finance Officer,International Rescue Committee (IRC),Yumbe,Contract,"Jan, 20",New,https://everjobs.ug/job/finance-officer-226/
4,Senior Lead - HR Shared Services,M-KOPA,Kampala,Full-time,"Jan, 20",New,https://everjobs.ug/job/senior-lead-hr-shared-...


In [12]:
fav = ['data', 'monitoring', 'analyst', 'M&E', 'teach']
loc = ['Lira', 'Gulu', 'Soroti']

df_list = []
for i in fav:
    data = df[(df['Job Title'].str.contains(i, case = False)) | (df['Location'].isin(loc))]
    df_list.append(data)

In [14]:
my_list = pd.concat(df_list)

In [16]:
my_list.head()

,Job Title,Company,Location,Job type,Date,Recency,Link
9,Request for Research Proposals Research Using ...,United Nations University,Kampala,Full-time,"Jan, 20",New,https://everjobs.ug/job/request-for-research-p...
26,Assistant Lecturer – Data Science and Artifici...,Cavendish University Uganda,Kampala,Part-time,"Jan, 20",New,https://everjobs.ug/job/assistant-lecturer-dat...
61,Establishing the Online Data Base of the Depar...,U.S. Agency for International Development (USAID),Kampala,Full-time,"Jan, 18",New,https://everjobs.ug/job/establishing-the-onlin...
79,"Vice President, Data Solutions",Bixal,Kampala,Full-time,"Jan, 18",New,https://everjobs.ug/job/vice-president-data-so...
84,Data Analyst and Visualization Senior Associat...,Bixal,Kampala,Full-time,"Jan, 18",New,https://everjobs.ug/job/data-analyst-and-visua...


In [24]:
my_list.drop_duplicates(subset = ['Job Title', 'Company', 'Location', 'Date'], inplace = True)

In [26]:
# my desired location
my_list['Location'].value_counts()

Location
Kampala    25
Soroti     11
Mityana     2
Gulu        1
Kyaka       1
Luuka       1
Mukono      1
Name: count, dtype: int64

In [30]:
my_list.to_excel('data_related.xlsx', index = False)

In [407]:
#df.to_excel('Jobs.xlsx')